In [1]:
import re 
import urllib.request 
from bs4 import BeautifulSoup 
 
def get_lyrics(artist,song_title): 
    artist = artist.lower() 
    song_title = song_title.lower() 
    # remove all except alphanumeric characters from artist and song_title 
    artist = re.sub('[^A-Za-z0-9]+', "", artist) 
    song_title = re.sub('[^A-Za-z0-9]+', "", song_title) 
    if artist.startswith("the"):    # remove starting 'the' from artist e.g. the who -> who 
        artist = artist[3:] 
    url = "http://azlyrics.com/lyrics/"+artist+"/"+song_title+".html" 
     
    try: 
        content = urllib.request.urlopen(url).read() 
        soup = BeautifulSoup(content, 'html.parser') 
        lyrics = str(soup) 
        # lyrics lies between up_partition and down_partition 
        up_partition = '<!-- Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that. -->' 
        down_partition = '<!-- MxM banner -->' 
        lyrics = lyrics.split(up_partition)[1] 
        lyrics = lyrics.split(down_partition)[0] 
        lyrics = lyrics.replace('<br/>','').replace('</br>','').replace('</div>','').strip() 
        return lyrics 
    except Exception as e: 
        return 0

### Lyrics Downloader

Use MusixMatch API to download and populate lyrics into the dataset.

In [2]:
import pandas as pd
# import numpy as np

# from lyrics_service import lyrics_finder

In [3]:
# skiprows used to ignore already downloaded lyrics
data = pd.read_csv('../data/train.csv') # ,skiprows=[i for i in range(1, 385)])
data.head()

,dzr_sng_id,valence,arousal,artist_name,track_name,lyrics,quadrant
0,213754,-0.682725,0.316758,Dark Tranquillity,Insanity's Crescendo,Gently hold our hands\nGently hold our heads o...,1
1,216752,-1.400403,1.538229,Anorexia Nervosa,Stabat mater dolorosa,We are the Sun\nWe are the dead stars\nWe are ...,1
2,254391,0.815393,0.662457,Uniting Nations,Out Of Touch,You're out of touch\nI'm out of time\nBut I'm ...,0
3,266313,1.257460,1.086515,The Walkmen,Brandy alexander,You finally close the door\nYou've left open w...,0
4,283306,0.373325,-0.923151,Duke Ellington,Caravan,随分先に行ってしまった 光の下のキャラバン\nトンネルに残響 塞いだ耳 自分嫌いな自分が好き...,3


In [4]:
track_name = data["track_name"]
artist_name = data["artist_name"]
track_name.shape, artist_name.shape

((10515,), (10515,))

In [ ]:
# list of downloaded lyrics
lyrics_list = []

for i in range(len(track_name)):
    response = get_lyrics(artist_name.iloc[i], track_name.iloc[i])
    
    if response == 0:
        lyrics_list.append(None)
    else:
        lyrics_list.append(response)
    
len(lyrics_list)

In [ ]:
data_copy = data.copy()

In [ ]:
lyrics_series = pd.Series(None, np.arange(len(track_name)))
lyrics_series.iloc[: len(lyrics_list)] = lyrics_list

In [ ]:
data_copy["lyrics full"][current:] = lyrics_series[:]

In [43]:
current = 385 - 1

In [44]:
data_copy["lyrics full"][current:] = lyrics_series[:]
data_copy["lyrics full"].iloc[current:]

<ipython-input-44-dc127d992fd5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation["Lyrics"][current:] = s[:]


384     Go on and wring my neck\nLike when a rag gets ...
385     Drought makes the workers dream\nMuscles and f...
386     We were so happy\nThings worked out better tha...
387     I know it sounds funny but I just can't stand ...
388     Melancholy\nAviation\nChocolate\nPerfume\nCiga...
                              ...                        
3858    On the fields of desolation\nWe are all alone\...
3859    Is it a sin, to love again?\nI cannot win, thi...
3860    Though you and I would dance and laugh and pla...
3861    Hello?\nIs there anybody in there?\nJust nod i...
3862    Money\nGet away\nYou get a good job with good ...
Name: Lyrics, Length: 3479, dtype: object

In [10]:
data_copy.to_csv("../data/train.csv", index=False)